In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import mpld3
#mpld3.enable_notebook()

## Comparison of new GC method for fragmented memory
Read in data from results.txt file.  
Has a description, 'Old Verion:', old data, 'New Version:', new data

In [ ]:
with open("../../wasm-of-ocaml/benchmarks/gc/results.txt") as f:
    old_data = []
    new_data = []
    new_with_threshold = []
    version = 0
    for line in f.readlines():
        if line.strip() == "":
            version += 1
        if line[0].isdigit():
            iters, mean, std = line.strip().split()
            if version == 0:
                old_data.append((int(iters), float(mean), float(std)))
            elif version == 1:
                new_data.append((int(iters), float(mean), float(std)))
            else:
                new_with_threshold.append((int(iters), float(mean), float(std)))

# reaches limiting behaviour after about 1000 iterations, so truncate data
#old_data = old_data[:-7]
#new_data = new_data[:-7]

In [ ]:
len(old_data)

In [ ]:
x = [x[0] for x in old_data]
y1 = np.array([x[1] for x in old_data])
y1err = np.array([x[2] for x in old_data])
y2 = np.array([x[1] for x in new_data])
y2err = np.array([x[2] for x in new_data])
y3 = np.array([x[1] for x in new_with_threshold])
y3err = np.array([x[2] for x in new_with_threshold])

# error bars are 1 std deviation
# Now that all tracing information left out, much closer performance
l1 = plt.errorbar(x, y1, yerr=y1err, label="old")
l2 = plt.errorbar(x, y2, yerr=y2err, label="new")
l3 = plt.errorbar(x, y3, yerr=y3err, label="new+threshold")
#plt.yscale("log")
plt.legend()

plt.figure()

plt.title("speedup")
plt.plot([],[])
plt.plot(x, y1/y2)
plt.plot(x, y1/y3)
#plt.ylim(0, 5)
#plt.ylim(1,np.max(y1/y2)*1.1)
#plt.legend([l1, l2, l3], ["old", "new", "new+threshold"])

In [ ]:
print(np.array(x))
print(np.array(y1))
print(np.array(y2))
print(np.array(y3))

## As above but for mergesort

In [ ]:
with open("../../wasm-of-ocaml/benchmarks/gc/mergesort_results.txt") as f:
    old_data = []
    new_data = []
    new_with_threshold = []
    version = 0
    for line in f.readlines():
        if line.strip() == "":
            version += 1
        if line[0].isdigit():
            iters, mean, std = line.strip().split()
            if version == 0:
                old_data.append((int(iters), float(mean), float(std)))
            elif version == 1:
                new_data.append((int(iters), float(mean), float(std)))
            else:
                new_with_threshold.append((int(iters), float(mean), float(std)))

In [ ]:
len(new_with_threshold)

## Benefit of cutoff for repeating GC
alltrees run with size 5, 6, 7, 8 and 9 (shadow stack overflow if tried with 10). Shows that old version and new version without threshold are near identical, but adding that threshold can dramatically reduce execution time. Execution time and memory usage of problem grows exponentially with tree size.

In [ ]:
with open("../../wasm-of-ocaml/benchmarks/gc/alltrees_results.txt") as f:
    old_data1 = []
    new_data1 = []
    new_with_threshold_data = [] # always grow heap if only 1KB (256 words) could be freed
    i = 0
    for line in f.readlines():
        if len(line.strip()) == 0:
            i += 1
        if line[0].isdigit():
            iters, mean, std = line.strip().split()
            if i == 0:
                old_data1.append((int(iters), float(mean), float(std)))
            elif i == 1:
                new_data1.append((int(iters), float(mean), float(std)))
            else:
                new_with_threshold_data.append((int(iters), float(mean), float(std)))

In [ ]:
x = [x[0] for x in old_data1]
y1 = np.array([x[1] for x in old_data1])
y1err = np.array([x[2] for x in old_data1])
y2 = np.array([x[1] for x in new_data1])
y2err = np.array([x[2] for x in new_data1])
y3 = np.array([x[1] for x in new_with_threshold_data])
y3err = np.array([x[2] for x in new_with_threshold_data])

# error bars are 1 std deviation
# Now that all tracing information left out, much closer performance
plt.errorbar(x, y1, yerr=y1err, label="Old")
plt.errorbar(x, y2, yerr=y2err, label="New")
plt.errorbar(x, y3, yerr=y3err, label="New + Threshold")
sizes = np.array(x)
plt.legend()
plt.xticks(x)
plt.yscale('log')

plt.figure()
plt.title("speedup")
plt.plot(x, y1/y2, label="New", c="tab:orange")
plt.plot(x, y1/y3, label="New + Threshold", c="g")
plt.xticks(x)
# mpld3 doesn't seem to work with hlines properly
#plt.hlines(1.0, 5, 9, color='b', linestyle='dotted')
plt.plot([5,10], [1,1], "--", label="Old")
plt.legend()
plt.show()

In [ ]:
print(x)
print(y1/y2)
print(y1/y3)

Tracing data from alltrees 9, showing number of frees each time GC is called. Note the length of the x-axis i.e. the total number of GC calls made.

In [ ]:
frees = []
memgrows = []
with open("../../wasm-of-ocaml/benchmarks/gc/trees9.txt") as f:
    f.readline()
    frees = eval(f.readline())[:-1]
    memgrows = eval(f.readline())[:-1]
    
#plt.figure(1,(0.03*len(frees),3))
plt.figure(1,(12,3))

plt.plot(frees, label="frees")
plt.vlines(np.where(memgrows), -300, -50, colors='k')
# don't show the large free of 2x as many objects at the end of each trace
plt.ylim(-200, 2200)
plt.title("Without threshold")

frees = []
memgrows = []
with open("../../wasm-of-ocaml/benchmarks/gc/newTrees9.txt") as f:
    f.readline()
    frees = eval(f.readline())[:-1]
    memgrows = eval(f.readline())[:-1]

# mark number of GC calls in optimised approach on the first plot
n = len(frees)
plt.plot([n,n],[-300,2200], "g--")
    
#plt.figure(2,(0.03*len(frees),3))
plt.figure(2,(12,3))

plt.plot(frees, "g", label="frees")
plt.vlines(np.where(memgrows), -300, -50, colors='k')
plt.ylim(-200, 2200)
plt.title("With threshold");
plt.show()

## More general results

In [ ]:
def normalise(means, errors, baseline, keep=False):
    new_means = {}
    new_errors = {}
    num_tests = len(means[baseline])
    for version in means:
        if version == baseline and not keep:
            continue
        new_means[version] = [means[version][i]/means[baseline][i] for i in range(num_tests)]
        if errors is not None:
            new_errors[version] =[errors[version][i]/means[baseline][i] for i in range(num_tests)]
    return new_means, new_errors

def bar_plot(ax, data, errors=None, colors=None, total_width=0.8, single_width=1, legend=False, capsize=3):
    """Draws a bar plot with multiple bars per data point.

    Parameters
    ----------
    ax : matplotlib.pyplot.axis
        The axis we want to draw our plot on.

    data: dictionary
        A dictionary containing the data we want to plot. Keys are the names of the
        data, the items is a list of the values.

        Example:
        data = {
            "x":[1,2,3],
            "y":[1,2,3],
            "z":[1,2,3],
        }

    errors: dictionary, optional
        Dictionary of standard deviations, corresponding structure to data

    colors : array-like, optional
        A list of colors which are used for the bars. If None, the colors
        will be the standard matplotlib color cyle. (default: None)

    total_width : float, optional, default: 0.8
        The width of a bar group. 0.8 means that 80% of the x-axis is covered
        by bars and 20% will be spaces between the bars.

    single_width: float, optional, default: 1
        The relative width of a single bar within a group. 1 means the bars
        will touch eachother within a group, values less than 1 will make
        these bars thinner.

    legend: bool, optional, default: True
        If this is set to true, a legend will be added to the axis.
    """

    # Check if colors where provided, otherwhise use the default color cycle
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # Number of bars per group
    n_bars = len(data)

    # The width of a single bar
    bar_width = total_width / n_bars

    # List containing handles for the drawn bars, used for the legend
    bars = []

    # Iterate over all data
    for i, (name, values) in enumerate(data.items()):
        # The offset in x direction of that bar
        x_offset = (i - n_bars / 2) * bar_width + bar_width / 2

        # Draw a bar for every value of that type
        for x, y in enumerate(values):
            if errors is None:
                bar = ax.bar(x + x_offset, y, width=bar_width * single_width, color=colors[i % len(colors)])
            else:
                err = errors[name][x]
                bar = ax.bar(x + x_offset, y, yerr=err, error_kw=dict(capsize=capsize),
                             width=bar_width * single_width, color=colors[i % len(colors)])

        # Add a handle to the last drawn bar, which we'll need for the legend
        bars.append(bar[0])

#    # Draw legend if we need
#    if legend:
#        ax.legend(bars, data.keys())
    # return the handlers/labels for a legend
    if legend:
        return bars, data.keys()

In [ ]:
data = {}
labels = ["New+Threshold", "None", "New", "Old"]
# Check file to see formatting
with open("../../wasm-of-ocaml/benchmarks/evaluation/ocaml_results.txt") as f:
    f.readline()
    for i in labels:
        f.readline()
        line = f.readline().strip().split()
        while line != []:
            if line[0] not in data:
                data[line[0]] = {}
            data[line[0]][i] = \
              {"time" : float(line[1]), "error" : float(line[2]), "heap": float(line[3]), "filesize" : float(line[4])}
            line = f.readline().strip().split()
tests = data.keys()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12,10))
labels = ["Old", "New", "New+Threshold"]
ax = axs[0]
times = {i: [data[test][i]["time"] for test in tests] for i in labels}
errors = {i: [data[test][i]["error"] for test in tests] for i in labels}
times, errors = normalise(times, errors, "Old", True)
#times = {lang : [means[lang][b] for b in benchmarks if b != "arith"] for lang in means if lang != "Grain"}
bar_plot(ax, times, errors=errors)
ax.set_title("Execution time")
ax.set_xticks([])
ax.set_ylabel("relative time")

ax = axs[1]
mem = {i: [data[test][i]["heap"] for test in tests] for i in labels}
mem, _ = normalise(mem, None, "Old", True)
handlers, labels = bar_plot(ax, mem, legend=True)
ax.set_title("Heap usage")
ax.set_xticks([])
#ax.set_yscale("log")
ax.set_ylabel("Memory (bytes)")

fig.legend(handlers, labels, loc='center left');
plt.subplots_adjust(left=0.15)

In [ ]:
tests

In [ ]:
labels = ["Old", "New", "New+Threshold"]
cases = [0, 3, 5, 6, 7, 9]
#print([list(tests)[i] for i in cases])
print("times = [" +  "; ".join([" ".join([str(times[label][i]) for label in labels]) for i in range(len(tests))]) + "];")
print("errors = [" +  "; ".join([" ".join([str(errors[label][i]) for label in labels]) for i in range(len(tests))]) + "];")
print()
print("mems = [" +  "; ".join([" ".join([str(mem[label][i]) for label in labels]) for i in range(len(tests))]) + "];")